In [ ]:
import os
import cv2
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

from utils.dataset import VideoDataset, MyConcatDataset
from utils.models import TrackNetV2MSE, TrackNetV2NLL
from utils.training import train_model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

%load_ext autoreload
%autoreload 2

# Dataset optimizations

In [ ]:
dataset_params = dict(image_size=(360, 640),
                      sequence_length=3,
                      sigma=5,
                      heatmap_mode='image',
                      one_output_frame=True,
                      drop_duplicate_frames=True,
                      transform = ToTensor(),
                      target_transform = ToTensor())

dataset_params['preload_in_memory'] = True
dataset = VideoDataset(root="../datasets/prova/", duplicate_equality_threshold=1, **dataset_params)

dataset_params['preload_in_memory'] = False
dataset_2 = VideoDataset(root="../datasets/prova/", duplicate_equality_threshold=1, **dataset_params)

Trying to find the bottleneck

In [ ]:
t = time.time()
for i in range(1000):
    dataset._generate_heatmap(np.random.randint(749, 780))
print(time.time()-t)

In [ ]:
t = time.time()
for i in range(1000):
    dataset._get_heatmap(np.random.randint(749, 780))
print(time.time()-t)

t = time.time()
for i in range(1000):
    dataset_2._get_heatmap(np.random.randint(749, 780))
print(time.time()-t)

Bottleneck is in the ToTensor() Transform

In [ ]:
t = time.time()
for i in range(100):
    dataset[np.random.randint(0, 100)]
print(time.time()-t)

t = time.time()
for i in range(10):
    dataset_2[np.random.randint(0, 100)]
print(time.time()-t)

In [ ]:
t = time.time()
for i in range(100):
    dataset[np.random.randint(0, 100)]
print(time.time()-t)

t = time.time()
for i in range(10):
    dataset_2[np.random.randint(0, 100)]
print(time.time()-t)

In [ ]:
t = time.time()
for i in range(100):
    dataset[np.random.randint(0, 100)]
print(time.time()-t)

t = time.time()
for i in range(10):
    dataset_2[np.random.randint(0, 100)]
print(time.time()-t)

In [ ]:
frames, labels = dataset[0]
frame1 = frames[0]

frames, labels = dataset[10]
frame2 = frames[0]

In [ ]:
t = time.time()
for i in range(1000):
    dataset._equal_frames(frame1, frame2)
print(time.time()-t)

In [ ]:
frames, labels = dataset[10]

In [ ]:
plt.imshow(frames[:3].numpy().transpose(1, 2, 0))

In [ ]:
plt.imshow(frames[:3].numpy().transpose(1, 2, 0))